In [22]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

In [23]:
tickers = pd.read_csv('prepared_csv/rusell1000_tickers.csv')
tickers = tickers["Name"].tolist()  

In [24]:
TRAIN_START_DATE = '2000-01-01'
TRAIN_END_DATE = "2018-09-31"

TRADE_START_DATE = "2018-10-01"
TRADE_END_DATE = '2024-12-31'


# TICKER_LIST = config_tickers.DOW_30_TICKER
TICKER_LIST = tickers

In [ ]:
df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = tickers).fetch_data()
    

In [ ]:
df_finrl.to_csv("prepared_csv/russel1000.csv", index=False)

In [ ]:
df_finrl = pd.read_csv("prepared_csv/russel1000.csv")

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_finrl)

processed
     


Successfully added technical indicators
YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (6287, 8)
Successfully added vix
Successfully added turbulence index


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2000-01-03,43.113319,47.267466,40.343885,47.155191,4674353,A,0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,43.113319,43.113319,24.209999,0.000000
1,2000-01-03,68.983864,71.221179,68.504440,70.741755,1291386,AA,0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,68.983864,68.983864,24.209999,0.000000
2,2000-01-03,0.650416,0.710640,0.650416,0.710640,1026865,AAON,0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,0.650416,0.650416,24.209999,0.000000
3,2000-01-03,0.839281,0.843498,0.762428,0.786328,535796800,AAPL,0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,0.839281,0.839281,24.209999,0.000000
4,2000-01-03,8.134053,8.366455,8.075953,8.192154,10635087,ABT,0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,8.134053,8.134053,24.209999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457845,2024-12-27,102.710396,104.166933,102.025525,102.536768,11943900,XOM,4,-3.009766,115.232736,98.542430,35.501189,-97.994771,35.534076,109.792470,112.570853,15.950000,46.218728
3457846,2024-12-27,18.318729,18.521840,18.183320,18.318729,1876900,XRAY,4,-0.327739,19.361053,17.681118,41.876158,-1.033591,12.792956,18.350530,20.218426,15.950000,46.218728
3457847,2024-12-27,133.289948,134.512164,132.363431,133.201242,1146300,YUM,4,-0.347982,139.452272,129.086104,50.438365,-12.751450,2.952786,133.779527,132.799088,15.950000,46.218728
3457848,2024-12-27,389.070007,394.700012,387.010010,393.369995,287200,ZBRA,4,-0.661513,418.459829,382.928177,50.900211,-67.555059,12.451864,398.143336,388.192167,15.950000,46.218728


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2000-01-03,A,43.113319,47.267466,40.343885,47.155191,4674353.0,0.0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,43.113319,43.113319,24.209999,0.000000
1,2000-01-03,AA,68.983864,71.221179,68.504440,70.741755,1291386.0,0.0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,68.983864,68.983864,24.209999,0.000000
2,2000-01-03,AAON,0.650416,0.710640,0.650416,0.710640,1026865.0,0.0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,0.650416,0.650416,24.209999,0.000000
3,2000-01-03,AAPL,0.839281,0.843498,0.762428,0.786328,535796800.0,0.0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,0.839281,0.839281,24.209999,0.000000
4,2000-01-03,ABT,8.134053,8.366455,8.075953,8.192154,10635087.0,0.0,0.000000,46.124176,36.809078,0.000000,-66.666667,100.000000,8.134053,8.134053,24.209999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019295,2024-12-27,XOM,102.710396,104.166933,102.025525,102.536768,11943900.0,4.0,-3.009766,115.232736,98.542430,35.501189,-97.994771,35.534076,109.792470,112.570853,15.950000,46.218728
5019296,2024-12-27,XRAY,18.318729,18.521840,18.183320,18.318729,1876900.0,4.0,-0.327739,19.361053,17.681118,41.876158,-1.033591,12.792956,18.350530,20.218426,15.950000,46.218728
5019297,2024-12-27,YUM,133.289948,134.512164,132.363431,133.201242,1146300.0,4.0,-0.347982,139.452272,129.086104,50.438365,-12.751450,2.952786,133.779527,132.799088,15.950000,46.218728
5019298,2024-12-27,ZBRA,389.070007,394.700012,387.010010,393.369995,287200.0,4.0,-0.661513,418.459829,382.928177,50.900211,-67.555059,12.451864,398.143336,388.192167,15.950000,46.218728


In [ ]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))
     


train.to_csv('prepared_csv/russel1000_train_data.csv')
trade.to_csv('prepared_csv/russel1000_trade_data.csv')

2593800
864050
